### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import nltk.data
import string
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix,classification_report
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

### Define useful functions

In [12]:
def text_process(review_sentence):
    return [word.lower() for word in review_sentence.split()]

### Import dataset

In [50]:
df_negative_sentences_tagged = pd.read_csv('./datasets/df_negative_sentences_part1_tagged.csv',lineterminator='\n')

### Drop NaN for review_topic

In [51]:
df_negative_sentences_tagged_subset = df_negative_sentences_tagged[df_negative_sentences_tagged['review_topic'].notna()]

In [52]:
df_negative_sentences_tagged_subset.groupby('review_topic').describe()

review_score_badge                                               \
                          count      mean       std  min   25%    50%    75%   
review_topic                                                                   
AH                         10.0  8.070000  1.369550  6.7  6.70   8.15   8.80   
B                          20.0  7.970000  1.388714  5.0  7.10   7.50   9.05   
BF                         22.0  8.204545  1.248419  5.4  7.50   8.00   9.00   
F                          11.0  8.309091  0.897167  7.1  7.95   8.30   8.55   
H                          29.0  8.134483  1.682531  2.5  7.50   8.00   9.20   
L                           8.0  8.200000  1.351190  5.4  7.50   8.80   9.00   
N                          23.0  7.752174  1.518932  5.0  6.70   7.50   9.10   
O                          39.0  9.115385  1.406851  2.5  8.55  10.00  10.00   
P                           4.0  9.200000  0.864099  8.0  8.90   9.40   9.70   
R                          42.0  7.904762  1.703075  2.5  7.10   8.15   9.15   
SM                          5.0  8.900000  1.523155  6.3  9.00   9.20  10.00   
ST                          7.0  7.728571  0.558911  7.1  7.30   7.50   8.30   
WF                          1.0  6.700000       NaN  6.7  6.70   6.70   6.70   

                   review_label                                     
               max        count mean  std  min  25%  50%  75%  max  
review_topic                                                        
AH            10.0         10.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
B             10.0         20.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
BF            10.0         22.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
F             10.0         11.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
H             10.0         29.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
L              9.6          8.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
N             10.0         23.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
O             10.0         39.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
P             10.0          4.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
R             10.0         42.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
SM            10.0          5.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
ST             8.3          7.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
WF             6.7          1.0  0.0  NaN  0.0  0.0  0.0  0.0  0.0

# Modeling

## Create model pipeline: Vectorization (BoW), TF-IDF, Naive Bayes Classifier

In [60]:
model_pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=text_process)),  # strings to token integer counts
    #('tfidf', TfidfTransformer()),  # integer counts to weighted TF-IDF scores
    ('classifier', MultinomialNB())  # train on TF-IDF vectors w/ Naive Bayes classifier
])

## Train-validation split

In [61]:
df_features = df_negative_sentences_tagged_subset.drop('review_topic', axis=1)

In [62]:
df_train, df_valid, label_train, label_valid = train_test_split(df_features, df_negative_sentences_tagged_subset['review_topic'], test_size=0.2)

## Train model

In [63]:
model_pipeline.fit(df_train['review_sentence'],label_train)

Pipeline(memory=None,
         steps=[('bow',
                 CountVectorizer(analyzer=<function text_process at 0x7f09db9c5040>,
                                 binary=False, decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=None, vocabulary=None)),
                ('classifier',
                 MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         verbose=False)

## Validate model

In [64]:
predictions = model_pipeline.predict(df_valid['review_sentence'])

## Print classification report

In [65]:
print(classification_report(label_valid,predictions))

              precision    recall  f1-score   support

          AH       0.00      0.00      0.00         2
           B       0.00      0.00      0.00         2
          BF       0.00      0.00      0.00         5
           F       0.00      0.00      0.00         2
           H       0.00      0.00      0.00         2
           L       0.00      0.00      0.00         1
           N       0.33      0.25      0.29         4
           O       0.71      0.56      0.63         9
           P       0.00      0.00      0.00         2
           R       0.41      0.85      0.55        13
          SM       0.00      0.00      0.00         1
          ST       0.00      0.00      0.00         2

    accuracy                           0.38        45
   macro avg       0.12      0.14      0.12        45
weighted avg       0.29      0.38      0.31        45



/home/roberto/anaconda3/envs/insight/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Print confusion matrix

In [66]:
valid_cm = confusion_matrix(label_valid,predictions)

In [67]:
valid_cm

array([[ 0,  0,  0,  0,  0,  0,  0,  0,  0,  2,  0,  0],
       [ 0,  0,  0,  0,  1,  0,  0,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  1,  0,  4,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  1,  0,  1,  0,  0],
       [ 0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  1,  0,  0,  3,  0,  0],
       [ 0,  0,  0,  0,  2,  0,  0,  5,  0,  2,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  1,  0,  0,  1,  0,  0],
       [ 0,  0,  0,  0,  2,  0,  0,  0,  0, 11,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  1,  0,  0,  0,  0,  0],
       [ 0,  0,  1,  0,  0,  0,  0,  0,  0,  1,  0,  0]])